In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from typing import Union
from tqdm.auto import tqdm as tqdm
from sklearn import preprocessing
import gc
import lightgbm as lgb
from hyperopt import hp
from tqdm import tqdm_notebook as tqdm
import random
import os
import re 
import lightgbm as lgb
import dask.dataframe as dd
from sklearn import preprocessing, metrics

In [2]:
data = pd.read_pickle('ca_model.pkl')
sample_submission = pd.read_csv('sample_submission.csv')

Like our other modeling and feature engineering notebooks, we will split our dataset by state.

In [3]:
ca_items = '[A-Z]+_\d{1}\d?_\d+_CA\w+'
sample_submission = sample_submission.loc[sample_submission['id'].str.contains(ca_items)]
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
'''Columns we can drop due to no importance'''

to_drop = ['is_quarter_start','is_quarter_end', 'is_year_start',
           'gap_placement', 'gap_size', 'gap_start',
           'number_of_gaps', 'is_month_start', 'is_year_end', 'is_month_end']

data.drop(to_drop, axis=1, inplace=True)

gc.collect()

60

In [4]:
features = [col for col in data.columns if col not in ['date','demand']]

x_train = data[data['date'] <= '2016-03-24']
y_train = x_train.sort_values('date')['demand']
x_test = data[(data['date'] > '2016-03-24')]
y_test = x_test['demand']
x_val = data[(data['date'] > '2016-03-24')]
y_val = x_val['demand']
test = data[(data['date'] > '2016-03-24')]
product = data[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
gc.collect()

20

In [2]:
# x_train = pd.read_pickle('x_train.pkl')
# y_train = pd.read_pickle('y_train.pkl')

In [29]:
train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

In [6]:
from scipy.sparse import csr_matrix

sales_train_val = pd.read_csv('sales_train_validation.csv')
product = sales_train_val[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
NUM_ITEMS = sales_train_val.shape[0]  # 30490
DAYS_PRED = sample_submission.shape[1] - 1  # 28

weight_mat = np.c_[np.ones([NUM_ITEMS,1]).astype(np.int8), # level 1
                   pd.get_dummies(product.state_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.item_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.item_id.astype(str),drop_first=False).astype('int8').values,
                   np.identity(NUM_ITEMS).astype(np.int8) #item :level 12
                   ].T


weight_mat_csr = csr_matrix(weight_mat)
del weight_mat; gc.collect()

def weight_calc(data,product):
    
    # calculate the denominator of RMSSE, and calculate the weight base on sales amount
    sales_train_val = pd.read_csv('sales_train_validation.csv')
    
    d_name = ['d_' + str(i+1) for i in range(1913)]

    sales_train_val = weight_mat_csr * sales_train_val[d_name].values

    # calculate the start position(first non-zero demand observed date) for each item / 商品の最初の売上日
    # 1-1914のdayの数列のうち, 売上が存在しない日を一旦0にし、0を9999に置換。そのうえでminimum numberを計算
    df_tmp = ((sales_train_val>0) * np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))

    start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1

    flag = np.dot(np.diag(1/(start_no+1)) , np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))<1

    sales_train_val = np.where(flag,np.nan,sales_train_val)

    # denominator of RMSSE / RMSSEの分母
    weight1 = np.nansum(np.diff(sales_train_val,axis=1)**2,axis=1)/(1913-start_no)

    # calculate the sales amount for each item/level
    df_tmp = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    df_tmp['amount'] = df_tmp['demand'] * df_tmp['sell_price']
    df_tmp =df_tmp.groupby(['id'])['amount'].apply(np.sum)
    df_tmp = df_tmp[product.id].values
    
    weight2 = weight_mat_csr * df_tmp 

    weight2 = weight2/np.sum(weight2)


    del sales_train_val
    gc.collect()
    
    return weight1, weight2

weight1, weight2 = weight_calc(data,product)


def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
    
          
    train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) / weight1) * weight2)
    
    return 'wrmsse', score, False

def wrmsse_simple(preds, data):
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
          
    train = np.c_[reshaped_preds, reshaped_true]
    
    weight2_2 = weight2[:NUM_ITEMS]
    weight2_2 = weight2_2/np.sum(weight2_2)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) /  weight1[:NUM_ITEMS])*weight2_2)
    
    return 'wrmsse', score, False

### HyperOpt Tuning

In [6]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from hyperopt import fmin, tpe, hp, anneal, Trials

random_state = 33
num_folds=3
n_iter=50

kf = KFold(n_splits=num_folds, random_state=random_state)

def gb_mse_cv(params, random_state=random_state, cv=kf, X=x_train, y=y_train):
    # the function gets a set of variable parameters in "param"
    params = {'n_estimators': int(params['n_estimators']), 
             'learning_rate': params['learning_rate']}
    
    # we use this params to create a new LGBM Regressor
    model = LGBMRegressor(random_state=random_state, **params)
    
    # and then conduct the cross validation with the same folds as before
    score = -cross_val_score(model, x_train, y_train, cv=cv, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    gc.collect()
    
    return score

In [ ]:
# possible values of parameters
space = {
    'boosting_type': 'gbdt',
    'n_estimators': hp.quniform('n_estimators', 100, 2000, 1),
    'learning_rate':    hp.choice('learning_rate', (0.05, 0.1, 0.5)),
    'bagging_frequency':    hp.choice('bagging_frequency', (8, 10, 12)),
    'bagging_fraction':    hp.choice('bagging_fraction', (0.7, 0.8)),
    'colsample_bytree': hp.choice('colsample_bytree', (0.5, 0.8)),
}

# trials will contain logging information
trials = Trials()

best=fmin(fn=gb_mse_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
         )

# computing the score on the test set
model = LGBMRegressor(random_state=random_state, n_estimators=int(best['n_estimators']),
                      learning_rate=best['learning_rate'])
model.fit(train_data,train_targets)
tpe_test_score=mean_squared_error(test_targets, model.predict(test_data))

gc.collect()
print("Best MSE {:.3f} params {}".format( gb_mse_cv(best), best))

  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

In [7]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization

In [18]:
categorical_feats = x_train.select_dtypes(include=['category'])

def bayes_parameter_opt_lgb(x_train, y_train, init_round=15, opt_round=25, n_folds=5,
                            random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=x_train, label=y_train, categorical_feature = categorical_feats, free_raw_data=False)
    
    def lgb_eval(learning_rate, bagging_freq, bagging_fraction, max_depth, subsample, colsample_bytree, min_child_weight):
        params = {'boosting_type':'gbdt','num_iterations':4000, 'early_stopping_round':100, 'metric':'custom', 'random_state':44}
        params["learning_rate"] = learning_rate
        params['bagging_freq'] = bagging_freq
        params['bagging_fraction'] = bagging_fraction
        params['max_depth'] = round(max_depth)
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample_bytree
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed,
                           stratified=True, verbose_eval =200, metric = ['rmse'], feval = wrmsse)
        return max(cv_result['rmse-mean'])

    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (.05, .1, .5),
                                        'bagging_freq': (8, 10, 12),
                                        'bagging_fraction': (0.7, 0.8),
                                        'subsample': (5, 8),
                                        'colsample_bytree': (0.5, 0.8)}, random_state=0)
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)

    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")

    # return best parameters
    return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(x_train, y_train, init_round=5, opt_round=10, n_folds=3, random_seed=6, n_estimators=100, learning_rate=0.05)


ValueError: setting an array element with a sequence.

In [19]:
def bayes_parameter_opt_lgb(x_train, y_train, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=x_train, label=y_train, categorical_feature = categorical_feats, free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(x_train, y_train, init_round=5, opt_round=10, n_folds=3, random_seed=6, n_estimators=100, learning_rate=0.05)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Instead of Bayes Opt, let's see how the HyperOpt package sorts through hyperperameter.

In [8]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, STATUS_FAIL
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}

lgb_fit_params = {
    'eval_metric': 'wrmsse',
    'early_stopping_rounds': 50
}
# model = lgb.train(lgb_reg_params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
#                   valid_sets = [train_set, val_set], verbose_eval = 100, feval= wrmsse)
# val_pred = model.predict(x_val[features])
    
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: wrmsse(pred, y_test)

In [ ]:
 gbm = lgb.train(params, 
                 lgb_train, 
                 num_boost_round=10, 
                 init_model=gbm, 
                 fobj=loglikelihood, 
                 feval=lambda preds, data:wrmsse(preds, data), 
                 valid_sets=lgb_eval) 
  
 print('Finished 50 - 60 rounds with self-defined objective function ' 
       'and multiple self-defined eval metrics...') 

In [ ]:
 print('Starting training with multiple custom eval functions...') 
 # train 
 gbm.fit(X_train, y_train, 
         eval_set=[(X_test, y_test)], 
         eval_metric=lambda preds, data:wrmsse(preds, data), 
         early_stopping_rounds=5) 